In [2]:
import csv
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import sklearn


In [18]:
l_0 = np.load("./class0.npy")
l_1 = np.load("./class1.npy")
l_2 = np.load("./class2.npy")
l_3 = np.load("./class3.npy")
l_4 = np.load("./class4.npy")
data = np.concatenate([l_0,l_1,l_2,l_3,l_4],axis=0)

In [2]:
label = pd.read_csv("./hand_rec/model/keypoint_classifier/keypoint.csv")

In [352]:
def to_onehot(labels,num_classes):
    out = np.zeros((labels.shape[0],num_classes))
    for i in range(labels.shape[0]):
        out[i][labels[i]]=1
    return out

In [353]:
to_onehot(label.iloc[:,0].values,4)

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [16]:
from torch.utils.data import Dataset
class CustomDataset_np(Dataset):
    def __init__(self, np_array ,num_class):
        self.data = np_array
        self.labels = torch.tensor(to_onehot(self.data[:,0],num_class),dtype=torch.float32)
        self.keypoints = torch.tensor(self.data[:,1:], dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        keypoint = self.keypoints[idx]
        label = self.labels[idx]
        return keypoint, label

In [17]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, data_dir,num_class):
        self.data = pd.read_csv(data_dir)
        self.labels = torch.tensor(to_onehot(self.data.iloc[:,0].values,num_class),dtype=torch.float32)
        self.keypoints = torch.tensor(self.data.iloc[:,1:].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        keypoint = self.keypoints[idx]
        label = self.labels[idx]
        return keypoint, label

In [387]:
dataset = CustomDataset("./hand_rec/model/keypoint_classifier/keypoint.csv",num_class = 4)

train_set, test_set = torch.utils.data.random_split(dataset, [4500,286])

In [388]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 16, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 16, shuffle = True)

In [365]:
class model(nn.Module):
    def __init__(self,num_class,dimension):
        super().__init__()
        self.linear1 = nn.Linear(21*dimension,2**(dimension+2))
        self.Dropout1 = nn.Dropout(p = 0.2)
        self.act1 = nn.ReLU()
        
        self.linear2 = nn.Linear(2**(dimension+2),2**(dimension+1))
        self.Dropout2 = nn.Dropout(p=0.2)
        self.act2 = nn.ReLU() 
        
        self.linear3 = nn.Linear(2**(dimension+1),num_class)
        self.act3 = nn.Sigmoid()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.Dropout1(out)
        out = self.act1(out)
        out = self.linear2(out)
        out = self.Dropout2(out)
        out = self.act2(out)    
        out = self.linear3(out)
        out = self.act3(out)  
        return out
    

In [366]:
model = model(num_class = 4, dimension=2)


In [369]:
dataset[:2][0].shape

torch.Size([2, 42])

In [394]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)
num_epochs = 500
loss_his = []

In [395]:
for epoch in range(num_epochs):
    running_loss = 0.0
    print("epoch : {}".format(epoch))
    for i,data in enumerate(train_loader):
        keypoint, labels = data
        optimizer.zero_grad()
        out = model(keypoint)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        loss_his.append(loss.item())
        running_loss += loss.item()
    print("loss : {}".format(running_loss/2000))
    
    

epoch : 0
loss : 0.13681248849630356
epoch : 1
loss : 0.13411218005418776
epoch : 2
loss : 0.13231270244717597
epoch : 3
loss : 0.130955115377903
epoch : 4
loss : 0.12912017971277237
epoch : 5
loss : 0.12801740768551825
epoch : 6
loss : 0.12665607756376265
epoch : 7
loss : 0.12686019802093507
epoch : 8
loss : 0.12585945242643357
epoch : 9
loss : 0.12522244960069656
epoch : 10
loss : 0.12449452367424965
epoch : 11
loss : 0.12377720412611962
epoch : 12
loss : 0.12438595876097679
epoch : 13
loss : 0.12381461897492409
epoch : 14
loss : 0.12338156804442406
epoch : 15
loss : 0.12276739105582238
epoch : 16
loss : 0.12283872270584106
epoch : 17
loss : 0.12192347338795662
epoch : 18
loss : 0.12217230638861656
epoch : 19
loss : 0.12199204462766647
epoch : 20
loss : 0.12165050315856933
epoch : 21
loss : 0.12150240302085877
epoch : 22
loss : 0.12122398462891579
epoch : 23
loss : 0.12104369819164276
epoch : 24
loss : 0.12119356942176819
epoch : 25
loss : 0.12076463425159455
epoch : 26
loss : 0.1211

loss : 0.11609177187085151
epoch : 215
loss : 0.11640275886654854
epoch : 216
loss : 0.11632002198696137
epoch : 217
loss : 0.11636139005422592
epoch : 218
loss : 0.11581166797876358
epoch : 219
loss : 0.11652548870444297
epoch : 220
loss : 0.11599770766496659
epoch : 221
loss : 0.11574007630348206
epoch : 222
loss : 0.11590111696720123
epoch : 223
loss : 0.11615439602732658
epoch : 224
loss : 0.11634614798426628
epoch : 225
loss : 0.11656222042441368
epoch : 226
loss : 0.11597798535227775
epoch : 227
loss : 0.115731030523777
epoch : 228
loss : 0.11606189385056495
epoch : 229
loss : 0.11564519584178924
epoch : 230
loss : 0.11586829659342766
epoch : 231
loss : 0.11596733275055886
epoch : 232
loss : 0.11603567746281623
epoch : 233
loss : 0.11619941210746765
epoch : 234
loss : 0.11586424562335014
epoch : 235
loss : 0.11612780615687371
epoch : 236
loss : 0.11597375720739364
epoch : 237
loss : 0.1162466560602188
epoch : 238
loss : 0.11604342487454414
epoch : 239
loss : 0.11585346186161041
e

loss : 0.11514386016130447
epoch : 426
loss : 0.1157033151090145
epoch : 427
loss : 0.1156308730840683
epoch : 428
loss : 0.11559819239377976
epoch : 429
loss : 0.11550440898537635
epoch : 430
loss : 0.11570457422733307
epoch : 431
loss : 0.11546311181783676
epoch : 432
loss : 0.11542404806613922
epoch : 433
loss : 0.11557286030054092
epoch : 434
loss : 0.11561955872178077
epoch : 435
loss : 0.11534306463599205
epoch : 436
loss : 0.11568868440389633
epoch : 437
loss : 0.11581133008003235
epoch : 438
loss : 0.11551529049873352
epoch : 439
loss : 0.1161838855445385
epoch : 440
loss : 0.11524823448061942
epoch : 441
loss : 0.11566397243738175
epoch : 442
loss : 0.11552339315414428
epoch : 443
loss : 0.11555601087212562
epoch : 444
loss : 0.11581608873605728
epoch : 445
loss : 0.11544306415319443
epoch : 446
loss : 0.11526512876152992
epoch : 447
loss : 0.1159176322221756
epoch : 448
loss : 0.1153529454767704
epoch : 449
loss : 0.11548063930869103
epoch : 450
loss : 0.11576635614037514
epo

In [407]:
pred_test = model(test_set[:][0])

In [413]:
correct = 0
for i in range(len(test_set)):
    if pred_test[i].argmax() == test_set[i][1].argmax() :
        correct = correct+1

In [414]:
correct

270

In [415]:
len(test_set)

286

In [417]:
pred_train = model(train_set[:][0])

In [418]:
correct = 0
for i in range(len(train_set)):
    if pred_train[i].argmax() == train_set[i][1].argmax() :
        correct = correct+1
accuracy = correct / len(train_set)

In [419]:
accuracy

0.9235555555555556

In [421]:
torch.save(model.state_dict(),"./test.pth")